<a href="https://colab.research.google.com/github/Frostbite22/learning-DL-by-doing-1/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import torch 
from torch import nn

In [64]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [65]:
from google.colab import drive
  
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/PyTorch/learning_by_doing1/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import zipfile 

with zipfile.ZipFile('archive.zip','r') as zip_ref:
  zip_ref.extractall()

In [69]:
path = os.getcwd()
path

'/content/drive/MyDrive/PyTorch/learning_by_doing1'

In [70]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('sign_mnist_train.csv')


In [71]:
train_df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [72]:
for index,row in train_df.iterrows():
  pixels = np.array(row[1:])
  print(pixels.reshape(28,28).shape)
  print(pixels)
  break

(28, 28)
[107 118 127 134 139 143 146 150 153 156 158 160 163 165 159 166 168 170
 170 171 171 171 172 171 171 170 170 169 111 121 129 135 141 144 148 151
 154 157 160 163 164 170 119 152 171 171 170 171 172 172 172 172 172 171
 171 170 113 123 131 137 142 145 150 152 155 158 161 163 164 172 105 142
 170 171 171 171 172 172 173 173 172 171 171 171 116 125 133 139 143 146
 151 153 156 159 162 163 167 167  95 144 171 172 172 172 172 172 173 173
 173 172 172 171 117 126 134 140 145 149 153 156 158 161 163 164 175 156
  87 154 172 173 173 173 173 173 174 174 174 173 172 172 119 128 136 142
 146 150 153 156 159 163 165 164 184 148  89 164 172 174 174 174 174 175
 175 174 175 174 173 173 122 130 138 143 147 150 154 158 162 165 166 172
 181 128  94 170 173 175 174 175 176 177 177 177 177 175 175 174 122 132
 139 145 149 152 156 160 163 165 166 181 172 103 113 175 176 178 178 179
 179 179 179 178 179 177 175 174 125 134 141 147 150 153 157 161 164 167
 168 184 179 116 126 165 176 179 180 180 1

In [84]:
from PIL import Image
from pathlib import Path 

#Setup path to a data folder 
data_path = Path(path)
image_path = data_path / "images"

#If the image folder doesn't exist, download it and prepare it 
if image_path.is_dir():
  print(f"{image_path} directory already exists")
else: 
  print(f"{image_path} does not exist, creating one")
  image_path.mkdir(parents=True,exist_ok=True)


counter = dict()

for index,row in train_df.iterrows(): 
    pixels = np.array(row[1:],dtype=np.uint8)
    pixels = np.reshape(pixels,(28,28))
    image = Image.fromarray(pixels)

    label = row[0]

    if label not in counter:
        counter[label] = 0
    counter[label] += 1

    filename = 'img{}_{}.png'.format(label, counter[label])
    image.save(image_path/filename)

    print('saved:', image_path/filename)


/content/drive/MyDrive/PyTorch/learning_by_doing1/images directory already exists
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img3_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img6_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img2_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img2_2.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img13_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img16_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img8_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img22_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img3_2.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img3_3.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img18_1.png
saved: /content/drive/MyDrive/PyTorch/learning_by_doing1/images/img10_1.png
saved: /conte

KeyboardInterrupt: ignored

In [82]:
img = Image.open('/content/drive/MyDrive/PyTorch/learning_by_doing1/images/img16_28.png')

In [83]:
img